**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
# !{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import re
import string
from IPython.display import HTML, display

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

phone_number_samples = [
    ("0903445772", (None, 903445772)),
    ("(541) 754-3010", (None, 5417543010)),
    ("554$117$22A", None),
    ("die Kartoffel", None),
    ("+1-541-754-3010", (1, 5417543010)),
    ("001-541-754-3010", (1, 5417543010)),
    ("+49-89-636-48018", (49, 8963648018)),
    ("+421 903 445 231", (421, 903445231)),
    ("4422-5588", (None, 44225588)),
    ("41 510 4405", (None, 415104405)),
    ("33 2187945", (None, 332187945)),
    ("+31 33 2187945", (31, 332187945)),
    ("(33) 445-88-76", (None, 334458876)),
    ("+65-2234-1487", (65, 22341487)),
    ("+65-XXXX-YYYY", None)
]

## Regulárne výrazy

Pri spracovaní textu je často potrebné vyhľadávať zhody s určitým kľúčovým slovom alebo vzorom. Často tiež potrebujeme také vzory nájsť a nahradiť. Tieto operácie sú veľmi jednoduché pokiaľ hľadáme jedno konkrétne kľúčové slovo. Ak však potrebujeme vykonať flexibilnejšie vyhľadávanie zahŕňajúce zložitejšie vzory, potrebujeme spôsob ako vyjadriť, čo hľadáme. Jeden spôsob ako to urobiť, je použiť regulárne výrazy, čo budeme ilustrovať v tomto notebook-u. V záujme stručnosti sa nebudeme venovať formálnemu úvodu do problematiky, ale priamo praktickým príkladom.

V notebook-u nepokryjeme celú syntax regulárnych výrazov – ďalšie informácie môžete nájsť napr. v: [Regular Expression HOWTO](https://docs.python.org/3/howto/regex.html) alebo v [re — Regular expression operations](https://docs.python.org/3/library/re.html).



In [4]:
#@title [A YouTube Video](https://youtu.be/rhzKDrUiJVk) { display-mode: "form" }
display(HTML("""
<iframe width="560" height="315" src="https://www.youtube.com/embed/rhzKDrUiJVk" frameborder="0"
allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture"
allowfullscreen></iframe>
"""))

### Jednoduché zhody

#### Explicitná zhoda: `keyword`

Ako prvý krok si ukážme, ako sa dá v Python-e vytvoriť náš prvý regulárny výraz. Aby sme veci nekomplikovali, budeme hľadať explicitnú zhodu, čo je to isté ako hľadať jednoducho podľa kľúčového slova. Budeme v texte (ktorý je tiež definovaný nižšie) hľadať slovo "the". Náš regulárny výraz bude jednoducho slovo, ktoré hľadáme: `the` a skompilujeme ho pomocou Python-ovej funkcie `re.compile`.



In [ ]:
text = ("The classic universal approximation theorem concerns " +
        "the capacity of feedforward neural networks.")

print(text)

In [ ]:
expr = re.compile("the")
list(expr.finditer(text))

Ako vidno, vyhľadávanie pomocou `expr.finditer` navrátilo interátor, ktorý sme transformovali na zoznam. Obsahuje celkovo dve zhody. Každá zhoda obsahuje jednak zhodujúci sa text a jednak jeho rozsah v pôvodnom texte.

#### Alternácia: `alternative1|alternative2`

Jedna vec, ktorú si môžeme v našom prvom príklade všimnúť, je, že prvé "the" v našom texte sa nezhodovalo, pretože to bolo v skutočnosti "The" a v regulárnych výrazoch sa malé a veľké písmená rozlišujú (aj keď to je možné zmeniť pomocou nepovinného parametra). Ak chceme nájsť zhodu s "the" aj "The", mohli by sme teda použiť operátor alternácie „|“ a napísať výraz, ktorý umožňuje dve alternatívne zhody: jednu pre "the" a druhú pre "The", tj `the|The`. Teraz budeme vedieť nájsť zhodu so všetkými tromi určitými členmi "the" v texte.



In [ ]:
text = ("The classic universal approximation theorem concerns " +
        "the capacity of feedforward neural networks.")

In [ ]:
expr = re.compile("the|The")
list(expr.finditer(text))

Môžeme tiež kombinovať štandardnú zhodu s alternatívami, takže by sme mohli povedať, že existuje alternatíva medzi "t" a "T" a zvyšok sa musí presne rovnať "he". Alternatívu medzi "t" a "T" bude potrebné uviesť v zátvorke: `(t|T)he`.



In [ ]:
text = ("The classic universal approximation theorem concerns " +
        "the capacity of feedforward neural networks.")

In [ ]:
expr = re.compile("(t|T)he")
list(expr.finditer(text))

#### Triedy znakov: `[cbaA]`, `[0-9a-zA-Z]`

Ešte iný spôsob ako dosiahnuť to isté, je špecifikovať prvý znak prostredníctvom triedy znakov (character class). Ak niekoľko znakov obalíme do hranatých zátvoriek, znamená to, že je povolená zhoda s ktorýmkoľvek z nich, t.j. v našom prípade: `[tT]he`.



In [ ]:
text = ("The classic universal approximation theorem concerns " +
        "the capacity of feedforward neural networks.")

In [ ]:
expr = re.compile("[tT]he")
list(expr.finditer(text))

Hoci sa toto môže javiť ako zbytočné cvičenie v uvádzaní rôznych syntaktických spôsobov ako vyjadriť ten istý koncept, triedy znakov sú v skutočnosti flexibilnejšie. Je napríklad možné vyjadriť pomocou nich aj určitý rozsah znakov. Aby sme napríklad pokryli celú abecedu, môžeme napísať `[a-zA-Z]`, čo pripúšťa zhodu so všetkými malými a veľkými písmenami. To isté by sme mohli spraviť aj s číslicami: `[0-9]`.

Ak by sme napríklad chceli nájsť zhodu so všetkými 2-písmenkovými kombináciami začínajúcimi na "n", mohli by sme napísať `n[a-zA-Z]`.



In [ ]:
text = ("The classic universal approximation theorem concerns " +
        "the capacity of feedforward neural networks.")

In [ ]:
expr = re.compile("n[a-zA-Z]")
list(expr.finditer(text))

#### Negatívne triedy znakov: `[^cbaA]`

Ak chceme znakovú triedu, ktorá sa bude zhodovať s čímkoľvek okrem danej množiny znakov, začneme triedu operátorom `^`. Napr. `[^e]` sa bude zhodovať s čímkoľvek iným než s "e". Ak teda napíšeme `n[^e]`, vylúčime tým zhody s "ne", ktoré sme vyššie dostali, ale získame zase zhody s "n ", pretože výraz teraz pripúšťa aj zhodu s bielymi znakmi.



In [ ]:
text = ("The classic universal approximation theorem concerns " +
        "the capacity of feedforward neural networks.")

In [ ]:
expr = re.compile("n[^e]")
list(expr.finditer(text))

#### Zhoda s bielymi znakmi: `\s`

Keď už hovoríme o bielych znakoch, zhodu s ľubovoľným bielym znakom vieme vyjadriť pomocou `\s`. Ak by sme teda chceli vylúčiť ešte aj zhody s "n " a ľubovoľnými inými bielymi znakmi, mohli by sme použiť `n[^e\s]`.



In [ ]:
text = ("The classic universal approximation theorem concerns " +
        "the capacity of feedforward neural networks.")

In [ ]:
expr = re.compile("n[^e\s]")
list(expr.finditer(text))

#### Zhoda s ľubovoľným znakom: `.`

Existuje aj notácia pre zhodu s akýmkoľvek znakom: `.`.



In [ ]:
text = ("The classic universal approximation theorem concerns " +
        "the capacity of feedforward neural networks.")

In [ ]:
expr = re.compile("ne.")
list(expr.finditer(text))

#### Rušenie meta znakov: `\.\^\+` a surové reťazce

Pri všetkých týchto špeciálnych znakoch by už malo byť zrejmé, že ak budeme chcieť nájsť zhodu s niektorým z nich, budeme musieť jeho špeciálny význam nejako vyrušiť (escape). Rušenie sa robí pomocou spätných lomiek `\`. Ak by sme teda chceli nájsť zhodu napríklad s doslovnou ".", regulárny výraz by vyzeral takto: `\.`. Ak však toto isté napíšeme v Python-e, v skutočnosti to nebude vždy správne fungovať, pretože `\` sa v Python-ových reťazcoch už používa na označenie špeciálnych znakov ako je znak nového riadka `\n`.

Aby sme teda do Python-ového reťazca zapísali skutočné spätnú lomku, museli by sme napísať dve spätné spätné lomky. Pre potreby regulárneho výrazu by sa potom správali ako jedna lomka. V prípade, že budeme lomiek potrebovať reťaziť viacej, taký zápis začne byť rýchlo neprehľadný. Našťastie, ak pred textový reťazec v Python-e zapíšeme `r`, napr. `r"\."`, indikuje to, že ide o špeciálny surový reťazec. Keď používame surové reťazce, nie je už potrebné používať namiesto každej spätnej lomky dve – môžeme náš regulárny výraz zapísať priamo.

Zoznam meta znakov, ktoré je potrebné rušiť ak ich chceme použiť v doslovnom význame nasleduje tu: `. ^ $ * + ? { } [ ] \ | ( )`.

Ak teda chceme vyhľadať napríklad dva ľubovoľné znaky nasledované bodkou, môžeme písať: `r"..\."`.



In [ ]:
text = ("The classic universal approximation theorem concerns " +
        "the capacity of feedforward neural networks.")

In [ ]:
expr = re.compile(r"..\.")
list(expr.finditer(text))

### Opakovanie: `+*{n}?`

Aby to bolo ešte zaujímavejšie, vieme tiež špecifikovať, či sa vzory smú opakovať a dokonca aj koľko ráz. Pozrime sa na niekoľko príkladov.

#### Presný počet opakovaní: `expr{n}`

Pridaním `{n}` na koniec výrazu (alebo podvýrazu, podľa potreby uzatvoreného v okrúhlych zátvorkách) špecifikujeme, že sa má opakovať presne `n` krát. Ak by sme teda cheli určiť, že hľadáme postupnosť ľubovoľných štyroch znakov okrem "e" a bielych znakov, mohli by sme písať `[^e\s]{4}`.

#### Ľubovoľný počet opakovaní: `expr*`

Ak chcem dovoliť ľubovoľný počet opakovaní, môžeme použiť operátor hviezdička `*`.

#### Jeden výskyt alebo viac: `expr+`

Aby sme vyjadrili, že sa má výraz vyskytnúť aspoň raz, ale smie sa vyskytnúť aj viac než raz, používame operátor `+`.

#### Nepovinný výraz: `expr?`

Aby sme vyjadrili, že je výraz nepovinný (smie sa vyskytnúť, ale nemusí), môžeme použiť operátor `?`.

#### Príklad: zhoda s ľubovoľným celým slovom

Povedzme, že by sme hľadali zhodu s ľubovoľným celým slovom, t.j. ľubovoľnou súvislou postupnosťou písmen oddelenou od svojho kontextu bielymi znakmi.



In [ ]:
sentence = "The classic universal approximation theorem."

In [ ]:
expr = re.compile(r"\s[a-zA-Z]+\s")
list(expr.finditer(sentence))

OK, zdá sa, že toto celkom nefunguje. To sa však pri písaní regulárnych výrazo stáva často: zabudneme na niektoré prípady, ktoré by mal výraz pokryť. Poďme to teda opraviť.

#### Vylúčenie medzier; lookbehind, lookahead: `(?<=...)`, `(?=...)`

Poďme najprv zo zhôd vylúčiť medzery. Je to potrebné spraviť, inak sa vzory pre susedné slová budú prekrývať a pri vyhľadávaní ich nenájdeme všetky.

* **Lookbehind:**  Aby sme vykonali porovnanie so vzorom na začiatku výrazu, ale ho nezahrnuli do nájdenej zhody, môžeme použiť tzv. lookbehind: `(?<=...)`, kde `...` nahradíme naším vzorom.


* **Lookahead:**  Podobne ak chceme porovnávať so vzorom na konci nášho výrazu, ale ho nechceme zahrnúť do nájdenej zhody, môžeme použiť lookahead: `(?=...)`.


Pre náš príklad by sme teda mohli medzery z nájdených zhôd vylúčiť takto: `(?<=\s)[a-zA-Z]+(?=\s)`. Týmto spôsobom by sme už mali byť schopní nájsť o jedno slovo viac.



In [ ]:
sentence = "The classic universal approximation theorem."

In [ ]:
expr = re.compile(r"(?<=\s)[a-zA-Z]+(?=\s)")
list(expr.finditer(sentence))

#### Použitie hraničného meta znaku

To už je o trochu lepšie, lenže nám stále chýba prvé a posledné slovo: pretože tieto nie sú ohraničené medzerami. Mohli by sme to riešiť použitím špeciálnych meta znakov pre koniec (`$`) a začiatok (`^`) reťazca a tiež explicitným spôsobom pridať všetky interpunkčné znamienka. Výsledný výraz by však už bol pomerne zložitý. Našťastie vieme to isté dosiahnuť pomocou **hraničného meta znaku**  `\b`, ktorý bude sledovať zhodu s hranicami slov:



In [ ]:
sentence = "The classic universal approximation theorem."

In [ ]:
expr = re.compile(r"\b[a-zA-Z]+\b")
list(expr.finditer(sentence))

#### Začiatok a koniec reťazca `^$`

Už sme spomenuli, že existujú meta znaky pre začiatok (`^`) a koniec (`$`) reťazca. Skúsme ich teda použiť na nájdenie prvého slova v reťazci. Použijeme jednoducho `^` nasledované vzorom, t.j.: `^[a-zA-Z]+`.



In [ ]:
sentence = "The classic universal approximation theorem."

In [ ]:
expr = re.compile(r"^[a-zA-Z]+")
list(expr.finditer(sentence))

#### Zhody s interpunkciou

Pri hľadaní zhôd s interpunkciou môžeme použiť reťazec `string.punctuation`, ktorý obsahuje všetky ASCII interpunkčné znamienka. Prirodzene, niektoré z týchto interpunkčných znamienok sa zároveň v regulárnych výrazoch používajú ako meta znaky, takže ich budeme potrebovať vyrušiť. To sa dá našťastie spraviť automaticky pomocou `re.escape`. Vytvorme si teda triedu znakov pre interpunkciu.



In [ ]:
sentence = "The classic? Universal; approximation. Theorem!"

In [ ]:
punct = string.punctuation
punct

In [ ]:
punct_class = "[" + re.escape(punct) + "]"
punct_class

In [ ]:
expr = re.compile(punct_class)
list(expr.finditer(sentence))

### Capture skupiny

Zhody, ktoré získame pomocou regulárnych výrazov môžu byť aj štruktúrované: namiesto toho, aby sme získali len úplný text zhody, vieme získať aj jednotlivé zložky, ak ich uzavrieme do capture skupín. Tieto sa vytvárajú pomocou okrúhlych zátvoriek. Napr. ak chceme nájsť zhodu s ľubovoľnými dvoma slovami nasledujúcimi za "the" a extrahovať každé z nich osobitne, môžeme písať: `[Tt]he ([a-zA-Z]+) ([a-zA-Z]+)`.



In [ ]:
sentence = "The classic universal approximation theorem."

In [ ]:
expr = re.compile(r"[Tt]he ([a-zA-Z]+) ([a-zA-Z]+)")
match = expr.search(sentence)
match

Keď sme získali zhodu `match`, môžeme sa pomocou `match.group(n)` odkázať na jej rôzne capture skupiny. Skupina 0 sa bude vždy odkazovať na celkovú zhodu.



In [ ]:
match.group(0)

Skupiny 1 a 2 budú v našom prípade zodpovedať prvému a druhému slovu.



In [ ]:
print(match.group(1))
print(match.group(2))

#### Necapture skupiny

Fakt, že okrúhle zátvorky slúžia na dva účely: na ohraničenie čiastkových výrazov a na označenie capture skupín, môže mať nepríjemné dôsledky. Ak by sme napríklad napísali regulárny výraz `(T|t)he ([a-zA-Z]+) ([a-zA-Z]+)`, skupina 1 by teraz zodpovedala prvej dvojici zátvoriek, ktorá ohraničuje alternatívu medzi `T` aa `t`.



In [ ]:
sentence = "The classic universal approximation theorem."

In [ ]:
expr = re.compile(r"(T|t)he ([a-zA-Z]+) ([a-zA-Z]+)")
match = expr.search(sentence)
match.group(1)

Takéto správenie často nie je žiaduce. V takých prípadoch vieme zmeniť supinu na necapture skupinu pomocou `(?:...)`. Náš regulárny výraz by teda v tomto prípade vyzeral takto: `(?:T|t)he ([a-zA-Z]+) ([a-zA-Z]+)`. Skupina 1 bude teraz korešpondovať so slovom "classic" pretože "T" už nie je v capture skupine.



In [ ]:
expr = re.compile(r"(?:T|t)he ([a-zA-Z]+) ([a-zA-Z]+)")
match = expr.search(sentence)
match.group(1)

### Nájsť a nahradiť

Okrem hľadania zhôd sa regulárne výrazy často používajú aj na hľadanie a nahrádzanie. Predpokladajme, že by sme napríklad výskyt každého určitého člena vo vete chceli nahradiť reťazcom `"XX"`. V Python-e môžeme použiť funkciu `expr.subn(repl, string)` na nahradenie všetkých výskytov výrazu v reťazci `string` reťazcom `repl`. Funkcia `subn` navracia výsledný reťazec a počet náhrad, ktoré sa vykonali. My si zobrazíme len výsledný reťazec.



In [ ]:
text = ("The classic universal approximation theorem concerns " +
        "the capacity of feedforward neural networks.")

In [ ]:
expr = re.compile(r"\b[tT]he\b")
print(expr.subn("XX", text)[0])

#### Použitie capture skupín v nahradzujúcom reťazci

Dajme tomu, že chceme realizovať o niečo komplikovanejšiu úlohu: napr. vymeniť medzi sebou prvé a posledné slovo začínajúce na "c". Samozrejme už vieme ako nájsť zhodu so slovami začínajúcimi na "c", ale ak ich chceme medzi sebou aj vymeniť, budeme musieť zachytiť prvé "c" slovo, posledné "c" slovo, text medzi nimi a vložiť ich naspäť v opačnom poradí. Našťastie platí, že v nahradzujúcom reťazci sa vieme odkázať späť na capture skupinu `n` pomocou výrazu `\n`. Takže jediné, čo musíme napísať je: `\b(c[a-zA-Z]+)\b` pre prvé "c" slovo, `(.*)` pre text medzi slovami a `\b(c[a-zA-Z]+)\b` pre posledné "c" slovo. Nahradzujúci reťazec bude jednoducho `\3\2\1`.



In [ ]:
text = ("The classic universal approximation theorem concerns " +
        "the capacity of feedforward neural networks.")

In [ ]:
expr = re.compile(r"\b(c[a-zA-Z]+)\b(.*)\b(c[a-zA-Z]+)")
print(expr.subn(r"\3\2\1", text)[0])

### Nerekurzívne a rekurzívne vzory

#### Príklad: odstránenie HTML tagov

V ďalšom príklade sa pokúsime odstrániť z textu všetky HTML tagy. Táto úloha by mala byť pomerne priamočiara: potrebujeme jednoducho nájsť zhodu so všetkým medzi znakmi `<` a `>`. Treba však pamätať ma to, že regulárne výrazy sú lačné (greedy) a budú sa snažiť skonzumovať maximálny možný počet znakov. Musíme byť preto pri špecifikácii výrazu opatrní. Pozrime sa, čo by sa stalo, keby sme náš regulárny výraz špecifikovali ako `<.*>`.



In [ ]:
text = """
text above
<div>
div 1 content
<span>inner span</span>
</div><div>
div 2 content
<span>inner span</span>
</div>
text below
"""

In [ ]:
expr = re.compile(r"<.*>")
print(expr.subn("", text)[0])

Ako vidno, tento regulárny výraz neodstránil len tagy: odstránil aj obsah vnútorného `<span>` tagu, čo sme nemali v úmysle. Správny výraz by bol `<[^>]*>`. V tomto prípade nebude možné nájsť zhodu, ktorá by prekračovala uzatvárajúcu zátvorku `>`.



In [ ]:
expr = re.compile(r"<[^>]*>")
print(expr.subn("", text)[0])

#### Regulárne výrazy nevedia vyjadriť rekurzívne vzory

Ďalšou možnou úlohou by bolo odstrániť HTML tagy vrátane ich obsahu. To sa však nedá urobiť len s použitím regulárnych výrazov: nie sú dostatočne expresívne na to, aby vedeli sledovať otváranie a uzatváranie tagov, pretože nedokážu vyjadriť rekurzívne vzory.

Na prácu so vzormi takého typu potrebujeme expresívnejšie jazyky a parsery: často založené na **bezkontextových gramatikách** .

---
### Úloha: telefónne čísla

**Na základe nižšie uvedených vzoriek vytvorte funkciu `match_number(sample)`, ktorá pomocou regulárnych výrazov vyhodnotí zhodu vzorky s telefónnym číslom. Ak je vstupný reťazec `sample` nepredstavuje validné telefónne číslo, funkcia navráti `None`. Ak predstavuje validné telefónne číslo, funkcia navráti dvojicu celých čísel predstavujúcich kód krajiny (ak nie je uvedený, namiesto toho `None`) a samotné telefónne číslo.** 

Samples of numbers with formats from [[apache.org](https://stdcxx.apache.org/doc/stdlibug/26-1.html),[wikipedia.org](https://en.wikipedia.org/wiki/National_conventions_for_writing_telephone_numbers)]:

* 754-3010: US, Local
* (541) 754-3010: US, Domestic
* +1-541-754-3010: US, International
* 001-541-754-3010: US, International
* +49-89-636-48018: German, International
* +421 903 445 231: Slovak, International
* 0903 445 231: Slovak, Domestic mobile
* 41 510 4405: Slovak, Domestic landline
* 4422-5588: Iceland, Domestic
* 33 2187945: Netherlands, Domestic
* +31 33 2187945: Netherlands, International
* (33) 445-88-76: Poland, Domestic
* +65-XXXX-YYYY: Singapore, International
---
Poznámky:

* Aby ste zabezpečili, že zhoda sa bude vyhodnocovať s celým reťazcom a nie len s jeho časťou, namiesto funkcie `search` alebo `match` použite funkciu `fullmatch`.
* Keď nájdete zhodu s číslom, zrejme z nej budete musieť pred konverziou na celé číslo pomocou funkcie `int` odstrániť znaky ako sú `'(', ')', '-'`. Nahradiť sa dajú napríklad pomocou `.replace` alebo `str.maketrans` a `.translate`.


In [ ]:
expr = re.compile( # ---

def match_number(sample):
    
    
    # ---
    
    

#### Testovanie

Teraz funkciu aplikujeme na niekoľko vzoriek a skontrolujeme výsledky.



In [ ]:
num_correct = 0

for sample, ret in phone_number_samples:
    try:
        retm = match_number(sample)
        
        if ret != retm:
            print("Incorrect response for sample '{}'.'".format(sample))
            print("  - Expected: '{}'".format(ret))
            print("  - Got: '{}'".format(retm))
        else:
            num_correct += 1
    except:
        print("Exception raised for sample '{}'.".format(sample))
        raise

print("{} correct out of {} samples".format(num_correct, len(phone_number_samples)))